<a href="https://colab.research.google.com/github/smenaaliaga/tesis_magister/blob/main/data_mimic_iv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de datos para tesis magister, MIMIC-IV

## Autentificación Google Colab

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Declaración de Project ID

In [ ]:
from google.cloud import bigquery

project_id = 'mimic-356201'
client = bigquery.Client(project=project_id)

## Ejecución de Querys

In [ ]:
import pandas as pd

In [ ]:
client.query('''
SELECT count(*) as total
FROM `physionet-data.mimiciv_hosp.admissions`
''').to_dataframe()

,total
0,454324


In [ ]:
client.query('''
SELECT count(*) as total
FROM `physionet-data.mimiciv_hosp.patients`
''').to_dataframe()

,total
0,315460


In [ ]:
diagnoses = client.query('''
SELECT *
FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
''').to_dataframe()

diagnoses.shape

(109775, 3)

In [ ]:
diagnoses.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [ ]:
diagnoses.groupby(['icd_version']).count()

,icd_code,long_title
icd_version,,
9,14666,14666
10,95109,95109


In [ ]:
diagnoses.loc[(diagnoses['icd_version'] == 9) & (diagnoses['icd_code'].isin(['99591', '99592', '78552'])), ['icd_code','long_title']]

,icd_code,long_title
9367,78552,Septic shock
12078,99591,Sepsis
12079,99592,Severe sepsis


In [ ]:
df = client.query('''
SELECT a.*, gender, anchor_age, anchor_year, anchor_year_group, dod
FROM `physionet-data.mimiciv_hosp.admissions` a
INNER JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` b
ON a.subject_id = b.subject_id AND a.hadm_id = b.hadm_id
INNER JOIN
(
  SELECT a.subject_id, a.hadm_id, min(c.seq_num) priority
  FROM `physionet-data.mimiciv_hosp.admissions` a
  INNER JOIN
  (
    SELECT subject_id, MAX(admittime) as max_admittime
    FROM `physionet-data.mimiciv_hosp.admissions`
    GROUP BY subject_id
  ) b ON a.subject_id = b.subject_id AND a.admittime = b.max_admittime
  INNER JOIN `physionet-data.mimiciv_hosp.diagnoses_icd` c
  ON a.subject_id = c.subject_id AND a.hadm_id = c.hadm_id
  WHERE c.icd_code in ('99591', '99592', '78552')
  GROUP BY a.subject_id, a.hadm_id
) c ON a.subject_id = c.subject_id AND a.hadm_id = c.hadm_id AND b.seq_num = c.priority
INNER JOIN `physionet-data.mimiciv_hosp.patients` d
ON a.subject_id = d.subject_id
''').to_dataframe()

df.shape

(3939, 20)

In [ ]:
df['admittime'] = pd.to_datetime(df['admittime']).dt.date
df['dischtime'] = pd.to_datetime(df['dischtime']).dt.date
df['deathtime'] = pd.to_datetime(df['deathtime']).dt.date

In [ ]:
df.loc[:, ['subject_id', 'admittime', 'dischtime', 'deathtime', 'dod']]

,subject_id,admittime,dischtime,deathtime,dod
0,10377695,2153-11-07,2153-11-07,2153-11-07,2153-11-07
1,10691939,2163-08-07,2163-08-07,NaT,None
2,12239697,2166-11-12,2166-11-15,NaT,2167-02-01
3,12652268,2169-07-03,2169-07-04,2169-07-04,2169-07-04
4,13110963,2128-05-06,2128-05-08,NaT,2128-05-08
...,...,...,...,...,...
3934,18601357,2155-04-22,2155-05-27,NaT,2155-06-03
3935,14723793,2149-05-01,2149-08-01,NaT,None
3936,16644289,2124-06-23,2124-08-14,NaT,2124-08-25
3937,17165725,2138-03-26,2138-04-02,NaT,2139-03-18


In [ ]:
import numpy as np

df2 = df
df['endtime'] = np.where(
    df2['dod'].notnull(), df2['dod'], np.where(
    df2['deathtime'].notnull(), df2['deathtime'], df2['dischtime'])) 

In [ ]:
df2.loc[:, ['subject_id', 'admittime', 'dischtime', 'deathtime', 'dod', 'endtime']]

,subject_id,admittime,dischtime,deathtime,dod,endtime
0,10377695,2153-11-07,2153-11-07,2153-11-07,2153-11-07,2153-11-07
1,10691939,2163-08-07,2163-08-07,NaT,None,2163-08-07
2,12239697,2166-11-12,2166-11-15,NaT,2167-02-01,2167-02-01
3,12652268,2169-07-03,2169-07-04,2169-07-04,2169-07-04,2169-07-04
4,13110963,2128-05-06,2128-05-08,NaT,2128-05-08,2128-05-08
...,...,...,...,...,...,...
3934,18601357,2155-04-22,2155-05-27,NaT,2155-06-03,2155-06-03
3935,14723793,2149-05-01,2149-08-01,NaT,None,2149-08-01
3936,16644289,2124-06-23,2124-08-14,NaT,2124-08-25,2124-08-25
3937,17165725,2138-03-26,2138-04-02,NaT,2139-03-18,2139-03-18


In [ ]:
df2['died'] = np.where(df2['dod'].isnull(), np.where(df2['deathtime'].isnull(), False, True), True)

In [ ]:
df2.loc[:, ['subject_id', 'admittime', 'dischtime', 'deathtime', 'dod', 'endtime', 'died']]

,subject_id,admittime,dischtime,deathtime,dod,endtime,died
0,10377695,2153-11-07,2153-11-07,2153-11-07,2153-11-07,2153-11-07,True
1,10691939,2163-08-07,2163-08-07,NaT,None,2163-08-07,False
2,12239697,2166-11-12,2166-11-15,NaT,2167-02-01,2167-02-01,True
3,12652268,2169-07-03,2169-07-04,2169-07-04,2169-07-04,2169-07-04,True
4,13110963,2128-05-06,2128-05-08,NaT,2128-05-08,2128-05-08,True
...,...,...,...,...,...,...,...
3934,18601357,2155-04-22,2155-05-27,NaT,2155-06-03,2155-06-03,True
3935,14723793,2149-05-01,2149-08-01,NaT,None,2149-08-01,False
3936,16644289,2124-06-23,2124-08-14,NaT,2124-08-25,2124-08-25,True
3937,17165725,2138-03-26,2138-04-02,NaT,2139-03-18,2139-03-18,True


In [ ]:
df2['survival_days'] = (df['endtime'] - df['admittime']).dt.days

In [ ]:
df2.loc[:, ['subject_id', 'admittime', 'endtime', 'survival_days', 'died']]

,subject_id,admittime,endtime,survival_days,died
0,10377695,2153-11-07,2153-11-07,0,True
1,10691939,2163-08-07,2163-08-07,0,False
2,12239697,2166-11-12,2167-02-01,81,True
3,12652268,2169-07-03,2169-07-04,1,True
4,13110963,2128-05-06,2128-05-08,2,True
...,...,...,...,...,...
3934,18601357,2155-04-22,2155-06-03,42,True
3935,14723793,2149-05-01,2149-08-01,92,False
3936,16644289,2124-06-23,2124-08-25,63,True
3937,17165725,2138-03-26,2139-03-18,357,True


In [ ]:
df3 = df2
df3 = df3[(df3['survival_days'] > 1) & (df3['anchor_age'] >= 16)]
df3.shape

(3605, 26)

In [ ]:
df3.loc[:, ['subject_id', 'survival_days', 'died']]

,subject_id,survival_days,died
2,12239697,81,True
4,13110963,2,True
5,13657153,7,False
7,14227792,3,False
8,14972735,6,False
...,...,...,...
3934,18601357,42,True
3935,14723793,92,False
3936,16644289,63,True
3937,17165725,357,True
